In [32]:
import numpy as np
from qiskit import *
from qiskit_optimization import QuadraticProgram
from qiskit.quantum_info import SparsePauliOp
from qiskit_optimization.converters import (InequalityToEquality, IntegerToBinary, 
                                            LinearEqualityToPenalty, LinearInequalityToPenalty,
                                            MaximizeToMinimize, QuadraticProgram2Ising)
# SPSA
from qiskit_optimization.spsa import minimize_spsa

# Fulqrum
from qiskit_optimization.fulqrum import CompositeWorkflow, PropertySet

from docplex.mp.model import Model

# SciPy minimizer routine
from scipy.optimize import minimize

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Sampler, Session
from qiskit.result import QuasiDistribution

from qiskit.circuit.library import EfficientSU2, TwoLocal

In [3]:
service = QiskitRuntimeService()

In [33]:
def evaluate_quadratic_program(bitstring, program):
    constant = program.objective.constant
    linear_elements = program.objective.linear.to_dict()
    quadratic_elements = program.objective._quadratic.to_dict()

    # Flip string so 0th bit is 0th array element for easy math
    x = np.fromiter(list(bitstring[::-1]), dtype=int)
    
    sum = constant
    for element, val in linear_elements.items():
        sum += x[element] * val
    for element, val in quadratic_elements.items():
        sum += x[element[0]] * val * x[element[1]]
    return sum

In [34]:
from functools import wraps

def validate_output_type(func):     
    @wraps(func)
    def wrapper(*args, **kwargs):
        output_types = args[0].output_types
        out = func(*args, **kwargs)
        if not isinstance(out, output_types):
            raise TypeError(f'Output type not in {output_types}')
        return out
    return wrapper  

In [56]:
import numpy as np

numpy.ndarray

In [59]:
class EvaluateProgramSolution:
    """Evaluate solutions
    """
    def __init__(self, program=None):
        self.program = program
        self.input_types = (QuasiDistribution, )
        self.output_types = (tuple, )
        self.property_set = PropertySet()

    @validate_output_type
    def run(self, dist):
        if self.program is None:
            program = self.property_set['qubo-transformer']['final_output']
        best_val = np.inf
        best_bits = None
        for bitstring in dist.binary_probabilities():
            temp = evaluate_quadratic_program(bitstring, self.program)
            if temp < best_val:
                best_val = temp
                best_bits = bitstring
        best_bits = best_bits[::-1]
        out = (best_val, np.fromiter(best_bits, int))
        return out


class UnrollQUBOVariables():
    """Unroll QUBO variables to their original definitions
    """
    def __init__(self, workflow):
        self.workflow = workflow
        self.input_types = (tuple, )
        self.output_types = (np.ndarray, )
        self.property_set = PropertySet()

    @validate_output_type
    def run(self, solution):
        x = np.fromiter(solution[1], dtype=int)
        for converter in self.workflow.passes[::-1]:
            x = converter.interpret(x)
        return x

In [36]:
mdl = Model("docplex model")
x = mdl.binary_var("x")
y = mdl.integer_var(lb=-2, ub=2, name="y")
mdl.minimize(2 * y - x)
# Min-result should be x = 1, y = -2 => -5
#print(mdl.export_as_lp_string())

In [37]:
backend = service.get_backend('ibm_nazca')

In [38]:
qp = QuadraticProgram.from_docplex_mp(mdl)

In [50]:
qubo_transformer = CompositeWorkflow([InequalityToEquality(), # Transformation
                                      IntegerToBinary(), # Transformation
                                      LinearEqualityToPenalty(), # Transformation
                                      LinearInequalityToPenalty(), # Transformation
                                      MaximizeToMinimize(), # Transformation
                                     ], name='qubo-transformer', store_final_output=True)

In [40]:
#session = Session(backend=backend)
#cw = CompositeWorkflow([transformer,
#                        QuadraticProgram2Ising(), # Quantum part - to Hamiltonian
                        #QAOASolver(session), # Quantum part - execution on QPU
                        #EvaluateProgramSolution() # Quantum part - bit-string to program variables
#                        ])

In [51]:
qubo = qubo_transformer.run(qp)
hamiltonian = QuadraticProgram2Ising().run(qubo)

In [52]:
hamiltonian

SparsePauliOp(['IIZI', 'IZII', 'ZIII', 'IIIZ', 'IIII'],
              coeffs=[-1. +0.j, -2. +0.j, -1. +0.j,  0.5+0.j, -0.5+0.j])

In [43]:
np.linalg.eig(hamiltonian.to_matrix())[1][:,1]

array([0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])

In [53]:
#session = Session(backend=backend)
from qiskit.primitives import Estimator as QiskitEstimator
from qiskit.primitives import Sampler as QiskitSampler

#estimator = Estimator(session=session, options={"shots": int(1e4)})
#sampler = Sampler(session=session, options={"shots": int(1e4)})


def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    cost = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    return cost


ansatz = TwoLocal(hamiltonian.num_qubits, 'ry', 'cx', 'linear', reps=1)
x0 = 2*np.pi*np.random.random(size=ansatz.num_parameters)
estimator = QiskitEstimator(options={"shots": int(1e4)})
sampler = QiskitSampler(options={"shots": int(1e4)})
res = minimize_spsa(cost_func, x0, args=(ansatz, hamiltonian, estimator), maxiter=100)
# Assign solution parameters to ansatz
qc = ansatz.assign_parameters(res.x)
# Add measurements to our circuit
qc.measure_all()
# Sample ansatz at optimal parameters
samp_dist = sampler.run(qc, shots=int(1e4)).result().quasi_dists[0]
# Close the session since we are now done with it
#session.close()

In [60]:
sol = EvaluateProgramSolution(qubo).run(samp_dist)

In [61]:
UnrollQUBOVariables(qubo_transformer).run(sol)

array([ 1., -2.])